In [47]:
#importing basics libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# Modelling-
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split,GridSearchCV, RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import pickle

In [48]:
df_train = pd.read_csv('family_financial_and_transactions_data.csv')

In [71]:
df_train.head()

,Category,Amount,Income,Savings,Monthly Expenses,Loan Payments,Credit Card Spending,Dependents,Financial Goals Met (%),Day,Is_Weekend
0,Travel,409.12,113810,20234,5781,2422,2959,2,68,7,0
1,Travel,270.91,113810,20234,5781,2422,2959,2,68,16,0
2,Groceries,91.10,113810,20234,5781,2422,2959,2,68,17,0
3,Healthcare,198.23,113810,20234,5781,2422,2959,2,68,25,0
4,Education,206.42,113810,20234,5781,2422,2959,2,68,25,0


In [83]:
# Define the target (y) and features (X)
X = df_train.drop(columns=['Monthly Expenses'])  # Dropping the target column
y = df_train['Monthly Expenses']  # Target variable

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [85]:
X_train.head()

,Category,Amount,Income,Savings,Loan Payments,Credit Card Spending,Dependents,Financial Goals Met (%),Day,Is_Weekend
10315,Utilities,144.79,135518,13025,237,1534,1,53,24,0
4245,Travel,244.61,42156,14880,873,1631,1,89,26,1
3728,Healthcare,274.82,66421,37031,3145,1368,2,52,24,0
10884,Utilities,269.05,144867,4315,220,719,2,67,11,0
11573,Healthcare,106.00,145957,27634,1111,985,1,61,6,1


In [74]:
cat_features = X.select_dtypes(include="object").columns
num_features = X.select_dtypes(exclude="object").columns


In [86]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')
preprocessor = ColumnTransformer(
[("OneHotEncoder", oh_transformer, cat_features),
("StandardScaler", numeric_transformer, num_features)
])

In [87]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [88]:

# Example input data (Ensure it's in the right format: 1 row, multiple columns)
input_data = {
    'Category': 'Groceries',
    'Amount': 200.0,
    'Income': 120000,
    'Savings': 30000,
    'Loan Payments': 2000,
    'Credit Card Spending': 1500,
    'Dependents': 2,
    'Financial Goals Met (%)': 75,
    'Day':2,
    'Is_Weekend': 4
}


In [81]:
X_train

array([[ 0.        ,  0.        ,  0.        , ..., -0.2661997 ,
         0.89129614, -0.58785592],
       [ 0.        ,  0.        ,  0.        , ...,  1.26598295,
         1.11516495,  1.7010971 ],
       [ 0.        ,  0.        ,  0.        , ..., -0.30876033,
         0.89129614, -0.58785592],
       ...,
       [ 0.        ,  0.        ,  1.        , ..., -0.13851782,
        -0.78771992, -0.58785592],
       [ 0.        ,  0.        ,  0.        , ..., -1.58557921,
        -0.67578551, -0.58785592],
       [ 0.        ,  0.        ,  0.        , ...,  1.73414987,
        -0.2280479 , -0.58785592]])

In [101]:
preprocessor.transform(input_df)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [55]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [56]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Define Random Forest and XGBoost models
models = {
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBoost": xgb.XGBRegressor()
}

# Function to evaluate model performance
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, predicted)
    return mae, rmse, r2

# Iterate through models and train
for model_name, model in models.items():
    model.fit(X_train, y_train)  # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test dataset
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    # Print model performance
    print(f"---------------- {model_name} ----------------")
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    
    print('='*35)
    print('\n')


---------------- Random Forest Regressor ----------------
Model performance for Training set
- Root Mean Squared Error: 0.0000
- Mean Absolute Error: 0.0000
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.0000
- Mean Absolute Error: 0.0000
- R2 Score: 1.0000


---------------- XGBoost ----------------
Model performance for Training set
- Root Mean Squared Error: 0.2116
- Mean Absolute Error: 0.1510
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.2228
- Mean Absolute Error: 0.1591
- R2 Score: 1.0000




In [58]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

# Define the model
model = XGBRegressor()

# Define the parameter distribution
param_dist = {
    'max_depth': [3, 6, 9, 12],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 200],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'min_child_weight': [1, 3, 5]
}

# Apply RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, cv=3, scoring='neg_mean_squared_error')
random_search.fit(X_train, y_train)

print(f"Best hyperparameters: {random_search.best_params_}")


Best hyperparameters: {'subsample': 0.8, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 12, 'learning_rate': 0.1, 'colsample_bytree': 0.8}


In [59]:
# Initialize XGBoost with the best parameters from RandomizedSearchCV
best_xgb_model = xgb.XGBRegressor(
    subsample=0.8,
    n_estimators=200,
    min_child_weight=1,
    max_depth=12,
    learning_rate=0.1,
    colsample_bytree=0.8
)

# Train the best model
best_xgb_model.fit(X_train, y_train)

# Make predictions on both the training and test data
y_train_pred = best_xgb_model.predict(X_train)
y_test_pred = best_xgb_model.predict(X_test)

# Evaluate the model
train_mae, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
test_mae, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred)

# Print the results
print("XGBoost Performance with Best Parameters:")
print('Model performance for Training set')
print(f"- Root Mean Squared Error: {train_rmse:.4f}")
print(f"- Mean Absolute Error: {train_mae:.4f}")
print(f"- R2 Score: {train_r2:.4f}")

print('----------------------------------')

print('Model performance for Test set')
print(f"- Root Mean Squared Error: {test_rmse:.4f}")
print(f"- Mean Absolute Error: {test_mae:.4f}")
print(f"- R2 Score: {test_r2:.4f}")


XGBoost Performance with Best Parameters:
Model performance for Training set
- Root Mean Squared Error: 0.0014
- Mean Absolute Error: 0.0012
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.0014
- Mean Absolute Error: 0.0012
- R2 Score: 1.0000


In [61]:

# Save the preprocessor
import joblib
joblib.dump(preprocessor, 'preprocessor.pkl')
print("Preprocessor saved successfully!")

Preprocessor saved successfully!


In [65]:

# Save the model
joblib.dump(best_xgb_model, 'xgb_model.pkl')
print("Model saved successfully!")

Model saved successfully!


In [102]:
with open('preprocessor.pkl', 'rb') as preprocessor_file:
    preprocessor = pickle.load(preprocessor_file)


In [67]:
with open('xgb_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

In [99]:
data = {
    'Category': 'Groceries',
    'Amount': 200.0,
    'Income': 120000,
    'Savings': 30000,
    'Loan Payments': 2000,
    'Credit Card Spending': 1500,
    'Dependents': 2,
    'Financial Goals Met (%)': 75,
}